# BPZ RAIL - DP0.2

no bringing to memory yet

## Imports

### common libs

In [ ]:
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline 

### RAIL

RAIL is a LSST-DESC software created to process different algorithms used to calculate photometric redshift. Its main goal is to minimize impact that different infrastructures can cause on different algorithms, for that it unifyes in a modular code supporting different inputs that different algorithms needs and padronizing the output so that it can be a more fair comparison between their results.

Rail uses 4 principal libraries in its core: <br>
_tables_io_: for data manipulation as hdf5 files, fits, etc. <br>
_qp_: used to paremitrize data PDFs for metrics calculation. <br>
_ceci_: construct pipelines, produces a .yaml within the steps and configurations as threads. <br>
_pzflow_: creates a flow for data creation. <br>

#### Core.
Where the main functions are going to manage the data and files that the program creates. It works based in the behavioral chain of resposability pattern (https://refactoring.guru/pt-br/design-patterns/chain-of-responsibility), where you create a flux in the code, where there is a request related/processed by a class handler that decides to pass it foward or not according to what is defined. So for that, what bpz does is create a class request (eg: Inform_BPZ_lite) that has all the inputs/configurations and is handled by its class handler (BPZ_lite).

#### Creation.
Contain all the support for data creation, as degradors, data flow creation, Column remapping, etc. It creates .hdf5 files with the data that is being manipulated.

#### Estimation.
This is where the codes are defined and executed.  <br>
inform: this is where the PRIORS for template fitting are informed and the machine learning codes are trained. <br>
estimate: where the algorith is executed though the .evaluate() function.
The code is wrapped as a RAIL stage so that it can be run in a controlled way. Estimation code can be stored in a yaml file to be run as a ceci module.


#### Evaluation.
This step contais the metrics for performance of the estimated codes.
<br>
------
For installation instructions check the official documentation: https://lsstdescrail.readthedocs.io/en/latest/source/installation.html <br>
For Rail versions check: https://github.com/LSSTDESC/RAIL/releases

In [ ]:
import rail
import qp
import tables_io

from rail.core.data import TableHandle
from rail.core.stage import RailStage
from rail.core.utilStages import ColumnMapper, TableConverter

##from rail.creation.engines.flowEngine import FlowEngine, FlowPosterior

from rail.estimation.algos.bpz_lite import Inform_BPZ_lite, BPZ_lite

from rail.evaluation.evaluator import Evaluator

#for rail versions
help(rail)

### LSST - TAP 

For accessing the data avaliable vis rubin science plataform we are going to use TAP.

TAP is a protocol created to access general table data. 
It uses html and xml to configure and acess the data, wich can be tabular, with key values that are stored in tabbles, one column per keyword, and non tabular such as images, an n-dimensional data. 
Also, it passes as parameters atributes that are configurable, for example, the language and the query that we want trough:

LANG=ADQL<br>
QUERY=< ADQL query string >

```xml
    <capability standardID="ivo://ivoa.net/std/TAP"> 
        <!-- BasicAA authentication bundle -->
        <interface xsi:type="urx:Async" role="std" version="1.1">
          <accessURL use="base">https://example.net/myTAP/auth-async</accessURL>
          <securityMethod standardID="ivo://ivoa.net/sso#BasicAA"/>
        </interface>
        <interface xsi:type="urx:Sync" role="std" version="1.1">
          <accessURL use="base">https://example.net/myTAP/auth-sync</accessURL>
          <securityMethod standardID="ivo://ivoa.net/sso#BasicAA"/>
        </interface>
     </capability>
```
By default it returns a TapResult, witch is a wrapper for the Astropy Table that constains some metadata of the schema that is being stored, that can be accessed by some methods as getColumn(), getRecords(), etc.

Its important to remember that TAP is a protocol to access the database where data is being stored, not the database itself.

TAPResults documentation: https://pyvo.readthedocs.io/en/latest/api/pyvo.dal.TAPResults.html <br>
Oficial documentation: https://www.ivoa.net/documents/TAP/ <br>
video 1: https://www.youtube.com/watch?v=hFmhypXg7JA&list=PL7kL5D8ITGyXDJYyms0rjzt9o-wDg-rKQ <br>
video 2:https://www.youtube.com/watch?v=BX10AI0WgMA&list=PL7kL5D8ITGyXDJYyms0rjzt9o-wDg-rKQ&index=2 <br>
video 4:https://www.youtube.com/watch?v=szDdL7sqD68&list=PL7kL5D8ITGyXDJYyms0rjzt9o-wDg-rKQ&index=3 <br>

In [ ]:
from lsst.rsp import get_tap_service

In [ ]:
service = get_tap_service()

assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

##### Example of a query

In [ ]:
query = "SELECT * FROM tap_schema.schemas"
results = service.search(query)
print(type(results))
results.to_table()

## General Configs

Setting some default number of rows for pandas. So that it doesnt display all of them. 

In [ ]:
pd.set_option('display.max_rows', 20)

Defining some variables that will help us with directories. 

In [ ]:
CURR_DIR = os.getcwd()
RAIL_DIR = os.path.join(os.path.dirname(rail.__file__), '..')
CURR_DIR, RAIL_DIR

## Reading DP0.2 data

the catalog with columns for dp 0.2 data can https://dm.lsst.org/sdm_schemas/browser/dp02.html

In [ ]:
max_rec = 1000
use_center_coords = "62, -37"
use_radius = "1.0"

In [ ]:
bands = ['g', 'i', 'r', 'u', 'y', 'z']

mags = ""
for band in bands:
    mags+= f"scisql_nanojanskyToAbMag({band}_cModelFlux) AS mag_{band}_cModel, {band}_cModelFluxErr, "

columns_query = f"objectId, {mags}coord_ra, coord_dec "

for this quey there is *detect_isPrimary* wich means that the source has no children, so that is already the final object. (this explanation is not very clear, but ok) and *r_extendedness* that defines if the object is a star or a galaxy, being 1 for galaxies and 0 for point objects such as starts.

In [ ]:
query = "SELECT " + columns_query + \
        "FROM dp02_dc2_catalogs.Object " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND detect_isPrimary = 1 " + \
        "AND r_extendedness = 1 " + \
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) > 17.0 " + \
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) < 23.0 "
print(query)

In [ ]:
%%time
results = service.search(query, maxrec=max_rec)
print(type(results))
results = results.to_table()
print(type(results))
results_pd = results.to_pandas()
results_pd.info(memory_usage="deep")

In [ ]:
results_pd.head()

---

##  RAIL BPZ

### Core - Data Storage 

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

Basically Rail store data in a transient class DataStore, this class associate keys and products in a dictionary, so that when program need some step it has the functions that read, writes, and a data handlers.

A DataHandler basically is a class that act like a handler for some data. What it does is that it associates the data with a file and the tool to read the file. The DataStore stores those handlers and their files associated with a key. So that when the algorithms process they are can propperly read the file content.

In [ ]:
DS

In [ ]:
columns_remmap = {
"objectId": "id",
"coord_ra": "coord_ra",
"coord_dec": "coord_dec",
"mag_g_cModel": "mag_g_lsst",
"g_cModelFluxErr": "mag_err_g_lsst",
"mag_i_cModel": "mag_r_lsst",
"i_cModelFluxErr": "mag_err_r_lsst",
"mag_r_cModel": "mag_i_lsst",
"r_cModelFluxErr": "mag_err_i_lsst",
"mag_u_cModel": "mag_u_lsst",
"u_cModelFluxErr": "mag_err_u_lsst",
"mag_y_cModel": "mag_y_lsst",
"y_cModelFluxErr": "mag_err_y_lsst",
"mag_z_cModel": "mag_z_lsst",
"z_cModelFluxErr": "mag_err_z_lsst",
"detect_isPrimary": "detect_isPrimary"
}

col_remapper_train = ColumnMapper.make_stage(name='col_remapper_train', columns=columns_remmap)
table_conv_train = TableConverter.make_stage(name='table_conv_train', output_format='numpyDict')

results_remmaped = col_remapper_train(results_pd)
## the redshift value is required and it is going to come from other surveys 
results_remmaped.data["redshift"] = 1

train_data = table_conv_train(results_remmaped)

As we can see, ceci stages basically configures the name and some configuration, so that when the stage runs, it return a TableHander, such as a PqHandler, Hdf5Handle or FitsHandle. 

obs: For machine leaning algorithms if may be necessary to configure a flowHandler too.

In [ ]:
type(results_remmaped), type(train_data)

In [ ]:
DS

In [ ]:
test_table = tables_io.convertObj(train_data.data, tables_io.types.PD_DATAFRAME)
test_table.head()

Here we should have somewhere a redshift result from other surveys.

### PRIORS - Inform

In [ ]:
DS

observe what is happening with the aliases as we go

In [ ]:
columns_file = os.path.join(CURR_DIR, 'configs/bpz.columns')
inform_bpz = Inform_BPZ_lite.make_stage(
    name='inform_bpzlite', 
    input="inprogress_output_table_conv_train.hdf5", 
    model='trained_BPZ_output.pkl', ##não precisaria isso pro bpz
    hdf5_groupname='', 
    columns_file=columns_file
)
inform_bpz.config.to_dict()

In [ ]:
DS

In [ ]:
type(train_data)

In [ ]:
help(inform_bpz.inform)

In [ ]:
%%time
returned = inform_bpz.inform(train_data)

In [ ]:
type(returned)

In [ ]:
inform_bpz.config.to_dict()

In [ ]:
DS

___

## Posterior -> Estimate


In [ ]:
estimate_bpz = BPZ_lite.make_stage(
    name='estimate_bpz', 
    hdf5_groupname='', 
    columns_file=columns_file, 
    model=inform_bpz.get_handle('model'))
estimate_bpz.is_parallel()

In [ ]:
help(estimate_bpz.estimate)

In [ ]:
estimate_bpz.config.to_dict()

In [ ]:
bpz_estimated = estimate_bpz.estimate(train_data)

In [ ]:
estimate_bpz.config.to_dict()

In [ ]:
DS

In [ ]:
type(bpz_estimated)

In [ ]:
#help(bpz_estimated())
bpz_estimated().build_tables()

results_tables = tables_io.convertObj(bpz_estimated().build_tables()['ancil'], tables_io.types.PD_DATAFRAME)
results_tables

In [ ]:
test_data_orig = results_remmaped.data

evaluator = Evaluator.make_stage(name=f'bpz_eval', truth=test_data_orig)
result_dict = evaluator.evaluate(bpz_estimated, test_data_orig)

In [ ]:
type(result_dict)

In [ ]:
help(evaluator.evaluate)

In [ ]:
results_tables = tables_io.convertObj(result_dict.data, tables_io.types.PD_DATAFRAME)
results_tables.head()

___
## VOU MEXER AINDA - Resultado pz x spec-z

In [ ]:
zmode = bpz_estimated().ancil['zmode']

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(train_data()['redshift'],zmode,s=1,c='k',label='simple bpz mode')
plt.plot([0,3],[0,3],'r--');
plt.xlabel("true redshift")
plt.ylabel("bpz photo-z")

### PIPELINES CECI

In [ ]:
import ceci
pipe = ceci.Pipeline.interactive()
stages = [flow_engine_train, lsst_error_model_train, inv_redshift,
          line_confusion, quantity_cut, col_remapper_train, table_conv_train,
          flow_engine_test, lsst_error_model_test, col_remapper_test, table_conv_test,  
          inform_knn, inform_fzboost, inform_bpz, estimate_knn, 
          estimate_fzboost, estimate_bpz, point_estimate_test,
          naive_stack_test]
for stage in stages:
    pipe.add_stage(stage)

In [ ]:
pipe.initialize(dict(flow=flow_file), dict(output_dir='.', log_dir='.', resume=False), None)
pipe.save('bpz_pipeline.yml')